In [ ]:
# !sudo pip3 install keras==2.1.5 tensorflow==1.13.1 numpy pandas pillow opencv--python sklearn optuna scikit-image ray ray[tune]

In [4]:
#!/usr/bin/env python3
# coding: utf-8
from optkeras.optkeras import OptKeras
import optkeras
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna
STUDY = None
N_TRIALS = 3
EPOCHS = 3

optuna_csv = 'optuna.csv'

with open('training.pkl', 'rb') as f:
     train = pickle.load(f)
    
with open('testing.pkl', 'rb') as f:
     test = pickle.load(f)
        
with open('validation.pkl','rb') as f:
     val = pickle.load(f)
        
train_photos, train_labels = list(), list()
tp = list()
for file in train:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    train_photos.append(photo)
    train_labels.append(output)
train_photos = asarray(train_photos)
train_labels = asarray(train_labels)

test_photos, test_labels = list(), list()
for file in test:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    tp.append(photo)
    test_photos.append(photo)
    test_labels.append(output)
test_photos = asarray(test_photos)
test_labels = asarray(test_labels)

val_photos, val_labels = list(), list()
for file in val:
    if 'Cat' in file:
        output = 1.0
    else:
        output = 0.0
    photo = load_img(file)
    photo = img_to_array(photo)
    val_photos.append(photo)
    val_labels.append(output)
val_photos = asarray(val_photos)
val_labels = asarray(val_labels)

train_photos =train_photos.astype('float32')
test_photos= test_photos.astype('float32')
val_photos =val_photos.astype('float32')
train_photos /= 255
val_photos /= 255
test_photos /= 255


nb_classes = 2
epochs=3
batch_size =2

vgg16_model = VGG16(weights = 'imagenet', include_top = False)
x = vgg16_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(input = vgg16_model.input, output = predictions)

for layer in vgg16_model.layers:
    layer.trainable = False
    
model.compile(optimizer = 'rmsprop',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model_info = model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, 
                   verbose=1,validation_data=(val_photos,val_labels))

model.save('model_hpo.h5')

m= load_model("model_hpo.h5")
test_photos = test_photos.astype("float32")

results = m.evaluate(test_photos, test_labels, batch_size=1)
optkeras.optkeras.get_trial_default = lambda: optuna.trial.FrozenTrial(
            None, None, None, None, None, None, None, None, None, None, None)
study_name = "CatsAndDogs" + '_Simple'
ok = OptKeras(study_name=study_name,
              monitor='val_acc',
              direction='maximize')

checkpoint_path = "cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

def objective(trial):
    nb_classes = 2
    epochs=3
    batch_size =2
    optimizer_options = ["RMSprop", "Adam", "SGD"]

    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation=trial.suggest_categorical('activation', ['relu', 'linear']))(x)
    predictions = Dense(nb_classes, activation = 'softmax')(x)
    model = Model(input = vgg16_model.input, output = predictions)
    for layer in vgg16_model.layers:
        layer.trainable = False

    model.compile(optimizer = trial.suggest_categorical("optimizer", ["rmsprop", "Adam", "SGD"]),loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)
    model.save_weights(checkpoint_path.format(epoch=0))
    model.fit(x=train_photos, y=train_labels,batch_size=2 , epochs=epochs, callbacks=ok.callbacks(trial),
              verbose=ok.keras_verbose ,validation_data=(val_photos,val_labels))
    return ok.trial_best_value



import os
import signal
import sys
import time
import traceback

from pathlib import Path


CHECKPOINT_FILE = "saved_stated.txt"

i = 0
try:
    with open(CHECKPOINT_FILE, mode="r") as f:
        i = int(f.read())
            
    def SIGTERM_handler(signum, frame):
        with open(CHECKPOINT_FILE, mode="w") as f:
            f.write(str(i))
            
    signal.signal(signal.SIGTERM, SIGTERM_handler)
    for _ in range(i,N_TRIALS+1):
        ok.optimize(objective, timeout = 3*60)
        if i == N_TRIALS:
            break
        i += 1
        time.sleep(1)
        with open(CHECKPOINT_FILE, mode="w") as f:
            f.write(str(i))
            
except Exception as e:
    traceback.print_exc()

        
# ok.optimize(objective, timeout = 3*60)
import pandas as pd

pd.options.display.max_rows = 8 

pd.options.display.max_rows = 8

put_csv = pd.read_csv(ok.keras_log_file_path)
put_csv.to_csv(optuna_csv)

Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - ETA: 11 - ETA: 29 - ETA: 26 - ETA: 22 - ETA: 19 - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


Instructions for updating:
Use tf.cast instead.
Train on 432 samples, validate on 48 samples
Epoch 1/3


428/432 [============================>.] - ETA: 1:22 - loss: 1.2013 - acc: 0.0000e+0 - ETA: 1:08 - loss: 2.3978 - acc: 0.2500    - ETA: 1:04 - loss: 2.4664 - acc: 0.166 - ETA: 1:03 - loss: 2.2635 - acc: 0.125 - ETA: 1:01 - loss: 2.2457 - acc: 0.100 - ETA: 1:01 - loss: 2.0100 - acc: 0.166 - ETA: 1:00 - loss: 1.8186 - acc: 0.214 - ETA: 58s - loss: 1.6809 - acc: 0.250 - ETA: 57s - loss: 1.6037 - acc: 0.27 - ETA: 56s - loss: 1.5365 - acc: 0.25 - ETA: 56s - loss: 1.4624 - acc: 0.27 - ETA: 55s - loss: 1.3772 - acc: 0.33 - ETA: 54s - loss: 1.4293 - acc: 0.30 - ETA: 54s - loss: 1.3526 - acc: 0.35 - ETA: 53s - loss: 1.3356 - acc: 0.36 - ETA: 53s - loss: 1.2732 - acc: 0.40 - ETA: 52s - loss: 1.3014 - acc: 0.38 - ETA: 52s - loss: 1.2488 - acc: 0.41 - ETA: 51s - loss: 1.1918 - acc: 0.44 - ETA: 51s - loss: 1.2487 - acc: 0.42 - ETA: 51s - loss: 1.2107 - acc: 0.45 - ETA: 50s - loss: 1.1901 - acc: 0.45 - ETA: 50s - loss: 1.1768 - acc: 0.43 - ETA: 50s - loss: 1.1649 - acc: 0.43 - ETA: 49s - loss: 1.143

432/432 [==============================] - ETA: 0s - loss: 0.6570 - acc: 0.681 - 64s 148ms/step - loss: 0.6549 - acc: 0.6829 - val_loss: 0.4217 - val_acc: 0.8750
Epoch 2/3


...

430/432 [============================>.] - ETA: 55s - loss: 2.5357 - acc: 0.50 - ETA: 55s - loss: 1.6883 - acc: 0.50 - ETA: 54s - loss: 1.1613 - acc: 0.66 - ETA: 54s - loss: 1.1580 - acc: 0.62 - ETA: 53s - loss: 0.9609 - acc: 0.70 - ETA: 53s - loss: 0.8480 - acc: 0.75 - ETA: 53s - loss: 0.7492 - acc: 0.78 - ETA: 53s - loss: 0.7388 - acc: 0.75 - ETA: 53s - loss: 0.6797 - acc: 0.77 - ETA: 52s - loss: 0.6137 - acc: 0.80 - ETA: 52s - loss: 0.5919 - acc: 0.81 - ETA: 52s - loss: 0.5663 - acc: 0.83 - ETA: 51s - loss: 0.5267 - acc: 0.84 - ETA: 51s - loss: 0.4953 - acc: 0.85 - ETA: 51s - loss: 0.4643 - acc: 0.86 - ETA: 50s - loss: 0.4622 - acc: 0.84 - ETA: 50s - loss: 0.4440 - acc: 0.85 - ETA: 50s - loss: 0.4204 - acc: 0.86 - ETA: 50s - loss: 0.4474 - acc: 0.84 - ETA: 49s - loss: 0.4258 - acc: 0.85 - ETA: 49s - loss: 0.4070 - acc: 0.85 - ETA: 49s - loss: 0.4466 - acc: 0.81 - ETA: 48s - loss: 0.4295 - acc: 0.82 - ETA: 48s - loss: 0.4449 - acc: 0.81 - ETA: 48s - loss: 0.4325 - acc: 0.82 - ETA: 48

120/120 [==============================] - ETA: 22 - ETA: 18 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 

[I 2020-09-10 08:56:15,007] A new study created in memory with name: CatsAndDogs_Simple


[2020-09-10 08:56:15.011475] [OptKeras] Ready for optimization. (message printed as verbose is set to 1+)


NameError: name 'os' is not defined

In [15]:
import joblib
STUDY = joblib.load("study_checkpoint_" + "vgg" + ".pkl")

In [16]:
STUDY.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_optimizer,user_attrs__Datetime_epoch_begin,user_attrs__Datetime_epoch_end,user_attrs__Trial_num,user_attrs_acc,user_attrs_loss,user_attrs_val_acc,user_attrs_val_loss,state
0,0,0.875,2020-09-10 10:38:04.364084,2020-09-10 10:41:17.425286,0 days 00:03:13.061202,linear,rmsprop,2020-09-10 10:40:10.220480,2020-09-10 10:41:17.036755,0,0.840278,0.371861,0.875,0.281831,COMPLETE


In [14]:
import joblib
new1 = joblib.load("hpo_study_checkpoint1_" + "vgg" + ".pkl")
new1.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_activation,params_optimizer,user_attrs__Datetime_epoch_begin,user_attrs__Datetime_epoch_end,user_attrs__Trial_num,user_attrs_acc,user_attrs_loss,user_attrs_val_acc,user_attrs_val_loss,state
0,0,0.937500,2020-09-10 09:25:21.972578,2020-09-10 09:28:35.769124,0 days 00:03:13.796546,relu,Adam,2020-09-10 09:27:29.188768,2020-09-10 09:28:35.342965,0,0.902778,0.258454,0.937500,0.206545,COMPLETE
1,1,0.708333,2020-09-10 09:28:35.779395,2020-09-10 09:31:50.288052,0 days 00:03:14.508657,relu,Adam,2020-09-10 09:30:48.136385,2020-09-10 09:31:50.286750,1,0.865741,0.313100,0.708333,0.497141,COMPLETE
2,2,0.770833,2020-09-10 09:31:50.298797,2020-09-10 09:35:08.326404,0 days 00:03:18.027607,relu,SGD,2020-09-10 09:34:03.355682,2020-09-10 09:35:07.994272,2,0.715278,0.584149,0.770833,0.539775,COMPLETE
3,3,0.666667,2020-09-10 09:35:08.338973,2020-09-10 09:38:22.295187,0 days 00:03:13.956214,relu,SGD,2020-09-10 09:37:16.917617,2020-09-10 09:38:22.293684,3,0.687500,0.597572,0.666667,0.572754,COMPLETE
4,4,0.895833,2020-09-10 09:38:22.438172,2020-09-10 09:41:52.043182,0 days 00:03:29.605010,relu,rmsprop,2020-09-10 09:40:45.961300,2020-09-10 09:41:51.558353,4,0.844907,0.334146,0.895833,0.270747,COMPLETE
5,5,0.770833,2020-09-10 10:19:11.161222,2020-09-10 10:22:25.163843,0 days 00:03:14.002621,relu,rmsprop,2020-09-10 10:21:18.473568,2020-09-10 10:22:25.162490,5,0.842593,0.330920,0.770833,0.479238,COMPLETE


In [6]:
import pickle
from os import listdir
from numpy import asarray
from numpy import save
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np
from keras import layers
from keras.layers import Input,Dense,BatchNormalization,Flatten,Dropout,GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.utils import layer_utils
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model,load_model
from keras.optimizers import RMSprop

import optuna


N_TRAIN_EXAMPLES = len(train_photos)
N_VALID_EXAMPLES = len(val_photos)
BATCHSIZE = 1
CLASSES = 2
EPOCHS = 3
STUDY = None
N_TRIALS = 10
TAR_FILE = "basicnet_model.tar.gz"


def objective(trial):
    # Clear clutter from previous Keras session graphs.
#     clear_session()

    (x_train, y_train), (x_valid, y_valid) = (train_photos,train_labels),(val_photos,val_labels)
#     img_x, img_y = x_train.shape[1], x_train.shape[2]
#     x_train = x_train.reshape(-1, img_x, img_y, 1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
#     x_valid = x_valid.reshape(-1, img_x, img_y, 1)[:N_VALID_EXAMPLES].astype("float32") / 255
#     y_train = y_train[:N_TRAIN_EXAMPLES]
#     y_valid = y_valid[:N_VALID_EXAMPLES]
#     input_shape = (img_x, img_y, 1)
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(CLASSES, activation = 'softmax')(x)
    model = Model(inputs = vgg16_model.input, outputs = predictions)
    
    for layer in vgg16_model.layers:
        layer.trainable = False

    # We compile our model with a sampled learning rate.
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"]
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

from keras.applications.vgg16 import VGG16
def create_classifier(trial):
    
    # We optimize the numbers of layers and their units.
    vgg16_model = VGG16(weights = 'imagenet', include_top = False)
    x = vgg16_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(CLASSES, activation = 'softmax')(x)
    model = Model(inputs = vgg16_model.input, outputs = predictions)
    
    for layer in vgg16_model.layers:
        layer.trainable = False
        
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer=RMSprop(lr=lr), metrics=["accuracy"])
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')]

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        callbacks=my_callbacks,
        verbose=1
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]


def objective(trial):
    classifier = create_classifier(trial)

    classifier.train(input_fn=train_input_fn, steps=TRAIN_STEPS)

    eval_results = classifier.evaluate(input_fn=eval_input_fn)

    return float(eval_results["accuracy"])


In [7]:
study = optuna.study.create_study( direction='maximize')
study.optimize(objective, n_trials=1)

[I 2020-09-10 08:57:26,532] A new study created in memory with name: no-name-3ac9f081-8f85-4de9-9f2a-b71f869e6ae1
[W 2020-09-10 08:57:27,252] Trial 0 failed because of the following error: NameError("name 'tf' is not defined",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/site-packages/optuna/study.py", line 778, in _run_trial
    result = func(trial)
  File "<ipython-input-6-cb39e55214f5>", line 112, in objective
    classifier = create_classifier(trial)
  File "<ipython-input-6-cb39e55214f5>", line 91, in create_classifier
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=2),
NameError: name 'tf' is not defined


NameError: name 'tf' is not defined